In [1]:
import numpy as np 
import time
import random
import mujoco_py
import gym
import os

from mjremote import mjremote
from mujoco_py import MjSim

from gym_hand_sim.envs import mpl_thumb_grasp_env

import tensorflow as tf 
import tf_agents
from tf_agents.environments import suite_mujoco
from tf_agents.environments import tf_py_environment, batched_py_environment
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.utils import common

MAX_EPISODE_STEPS = 50

num_iterations = 20000 # @param {type:"integer"}
initial_collect_steps = 100  # @param {type:"integer"} 
collect_steps_per_iteration = 1  # @param {type:"integer"}
replay_buffer_max_length = 100000  # @param {type:"integer"}
batch_size = 64  # @param {type:"integer"}
learning_rate = 1e-3  # @param {type:"number"}
log_interval = 200  # @param {type:"integer"}
num_eval_episodes = 10  # @param {type:"integer"}
eval_interval = 1000  # @param {type:"integer"}
num_parallel_environments = 4

tf_agents.system.multiprocessing.enable_interactive_mode()
train_py_env = suite_mujoco.load('gym_hand_sim:MplThumbGraspBall-v0')
eval_py_env = suite_mujoco.load('gym_hand_sim:MplThumbGraspBall-v0')
train_env = tf_py_environment.TFPyEnvironment(batched_py_environment.BatchedPyEnvironment([train_py_env] * num_parallel_environments))
eval_env = tf_py_environment.TFPyEnvironment(eval_py_env)


running build_ext
c:\users\xieji\repos\gym-hand-sim\gym_hand_sim\envs
c:\users\xieji\repos\gym-hand-sim\gym_hand_sim\envs


In [2]:
def compute_avg_return(environment, policy, num_episodes=10):
    total_return = 0.0
    for _ in range(num_episodes):

        time_step = environment.reset()
        episode_return = 0.0

        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward
        total_return += episode_return
    avg_return = total_return / num_episodes
    return avg_return.numpy()[0]

def create_nets(observation_spec, action_spec):
    policy_net = ActorDistributionRnnNetwork(
        observation_spec,
        action_spec,
        input_fc_layer_params = (256,),
        lstm_size = (128,),
        output_fc_layer_params = (128,),
        activation_fn = tf.nn.relu)
    value_net = ValueRnnNetwork(
        observation_spec,
        action_spec,
        input_fc_layer_params = (256,),
        lstm_size = (128,),
        output_fc_layer_params = (128,),
        activation_fn = tf.nn.relu)
    
    return policy_net, value_net


In [ ]:


'''
Init policy (actor) network and value (critic) network

for k = 0,1,2:
    collect set of trajectories {tau_i} by running pi_k = pi(theta_k) in env
    compute rewards R_t_hat
    compute advantage estimates A_t_hat based on current value function V_phi_k
    update policy (pi(theta_k+1))via PPO-Clip - SGA w/ Adam
    fit value function (V_phi_k+1) by regression
'''

In [4]:
time_step = train_py_env.reset()

#??
fc_layer_params = (100,100)

q_net = q_network.QNetwork(
    train_env.observation_spec(),
    train_env.action_spec(),
    fc_layer_params=fc_layer_params)

optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)

train_step_counter = tf.Variable(0)

agent = dqn_agent.DqnAgent(
    train_env.time_step_spec(),
    train_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter)

agent.initialize()
eval_polivy = agent.policy
collect_policy = agent.collect_policy

random_policy = random_tf_policy.RandomTFPolicy(train_env.time_step_spec(),
                                                train_env.action_spec())

compute_avg_return(eval_env, random_policy)
""""
while True:
    next_time_step = train_py_env.step(np.array([1.,1.,1.,1.]))
    train_py_env.render()
    print(next_time_step)"""

NameError: name 'agent' is not defined

In [6]:
print(eval_env.action_spec())

BoundedTensorSpec(shape=(4,), dtype=tf.float32, name='action', minimum=array(-1., dtype=float32), maximum=array(1., dtype=float32))
